In [1]:
pip install --upgrade google-cloud-videointelligence

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='extreme-zephyr-381810-6442a502a8b5.json'

In [3]:
from google.cloud import videointelligence

In [ ]:
# Load the labels for each sample from a CSV file
if is_train == True:
    dataFrame = pd.read_csv('train.csv').values[:, 1:]
else:
    dataFrame = pd.read_csv('test.csv').values[:, 1:]
self.samples = []
for dataPath, label in tqdm(dataFrame):
    if not os.path.isdir(os.path.join(dataPath)):
        continue

    video_path = None
    text_path = None
    for filename in os.listdir(os.path.join(dataPath)):
        if filename.endswith('.mp4'):
            video_path = os.path.join(dataPath, filename)
        elif filename.endswith('.txt'):
            text_path = os.path.join(dataPath, filename)
    if video_path is None or text_path is None:
        continue
    video_path.append(video_path)

# Video

In [5]:
import io

In [6]:
"""Detect labels given a file path."""
video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.Feature.LABEL_DETECTION]
video=[]
for i in range(len(video_path)):
    z1=[]
    path=video_path[i]

    with io.open(path, "rb") as movie:
        input_content = movie.read()

    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content}
    )
    ##print("\nProcessing video for label annotations:")

    result = operation.result(timeout=90)
    ##print("\nFinished processing.")

    # Process video/segment level label annotations
    segment_labels = result.annotation_results[0].segment_label_annotations
    for i, segment_label in enumerate(segment_labels):
        ##print("Video label description: {}".format(segment_label.entity.description))
        a0="{}".format(segment_label.entity.description)
        for category_entity in segment_label.category_entities:
            #print("\tLabel category description: {}".format(category_entity.description) )
            
            a1="{}".format(category_entity.description)
        for i, segment in enumerate(segment_label.segments):
            start_time = round((
                segment.segment.start_time_offset.seconds
                + segment.segment.start_time_offset.nanos / 1e9
            ),2)
            end_time = round((
                segment.segment.end_time_offset.seconds
                + segment.segment.end_time_offset.nanos / 1e9
            ),2)
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = round(segment.confidence,2)
            ##print("\tSegment {}: {}".format(i, positions))
            ##print("\tConfidence: {}".format(confidence))
        ##print("\n")
        if confidence>0.8:
            z1.append(a0)
            z1.append(a1)
            #z1.append(positions)
            #z1.append(confidence)
        else:
            pass

    # Process shot level label annotations
    shot_labels = result.annotation_results[0].shot_label_annotations
    for i, shot_label in enumerate(shot_labels):
        ##print("Shot label description: {}".format(shot_label.entity.description))
        a0="{}".format(shot_label.entity.description)
        for category_entity in shot_label.category_entities:
            ##print("\tLabel category description: {}".format(category_entity.description))
            a1="{}".format(category_entity.description)


        for i, shot in enumerate(shot_label.segments):
            start_time = round((
                shot.segment.start_time_offset.seconds
                + shot.segment.start_time_offset.nanos / 1e9
            ),2)
            end_time = round((
                shot.segment.end_time_offset.seconds
                + shot.segment.end_time_offset.nanos / 1e9
            ),2)
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = round(shot.confidence,2)
            ##print("\tSegment {}: {}".format(i, positions))
            ##print("\tConfidence: {}".format(confidence))
        ##print("\n")
        z1.append(a0)
        z1.append(a1)
        #z1.append(positions)
        #z1.append(confidence)

    # Process frame level label annotations
    frame_labels = result.annotation_results[0].frame_label_annotations
    for i, frame_label in enumerate(frame_labels):
        ##print("Frame label description: {}".format(frame_label.entity.description))
        '''for category_entity in frame_label.category_entities:
            print("\tLabel category description: {}".format(category_entity.description))'''

        # Each frame_label_annotation has many frames,
        # here we print information only about the first frame.
        frame = frame_label.frames[0]
        time_offset = frame.time_offset.seconds + frame.time_offset.nanos / 1e9
        ##print("\tFirst frame time offset: {}s".format(time_offset))
        ##print("\tFirst frame confidence: {}".format(frame.confidence))
        ##print("\n")
    video.append(z1)
        

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhangzijian/Desktop/Learning/bda semester1/IS6912 is project /Googleai/q11.ts'

In [173]:
video

[['black and white',
  'style',
  'clip art',
  'graphics',
  'display device',
  'technology',
  'food',
  'technology',
  'graphics',
  'artwork',
  'audio',
  'technology',
  'finger',
  'hand',
  'car',
  'vehicle',
  'space',
  'vehicle',
  'font',
  'software',
  'hand',
  'software',
  'motor vehicle',
  'vehicle',
  'computer wallpaper',
  'software',
  'land vehicle',
  'vehicle',
  'art',
  'vehicle',
  'brand',
  'vehicle',
  'text',
  'vehicle',
  'vehicle',
  'vehicle',
  'multimedia',
  'vehicle',
  'computer program',
  'software',
  'headphones',
  'technology',
  'song',
  'music',
  'lyrics',
  'music',
  'symbol',
  'information',
  'audio equipment',
  'audio',
  'trademark',
  'audio',
  'beats electronics',
  'audio',
  'led backlit lcd display',
  'technology',
  'graphic design',
  'artwork',
  'review',
  'artwork',
  'logo',
  'graphics'],
 ['machine',
  'weather',
  'learning',
  'person',
  'weather',
  'person',
  'rain and snow mixed',
  'weather',
  'wint

In [174]:
with open("video.txt", "w") as f:
    for item in video:
        f.write("%s\n" % item)

# speech

In [18]:
import io
import os
from google.cloud import videointelligence_v1p3beta1 as videointelligence


In [194]:
"""Transcribe speech from a video stored on GCS."""
from google.cloud import videointelligence
speech=[]
for i in range(len(video_path)):
    z=[]
    path=video_path[i]
    with io.open(path, 'rb') as file:
        input_content = file.read()

    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.SPEECH_TRANSCRIPTION]

    config = videointelligence.SpeechTranscriptionConfig(
        language_code="en-US", enable_automatic_punctuation=True
    )
    video_context = videointelligence.VideoContext(speech_transcription_config=config)

    operation = video_client.annotate_video(
        request={
            "features": features,
            "input_content": input_content,
            "video_context": video_context,
        }
    )

    print("\nProcessing video for speech transcription.")

    result = operation.result(timeout=600)

    # There is only one annotation_result since only
    # one video is processed.
    annotation_results = result.annotation_results[0]
    for speech_transcription in annotation_results.speech_transcriptions:

        # The number of alternatives for each transcription is limited by
        # SpeechTranscriptionConfig.max_alternatives.
        # Each alternative is a different possible transcription
        # and has its own confidence score.
        for alternative in speech_transcription.alternatives:
            print("Alternative level information:")
            print("Transcript: {}".format(alternative.transcript))
            print("Confidence: {}\n".format(alternative.confidence))
            if alternative.confidence>0.8:
                z.append(alternative.transcript)
                #z.append(alternative.confidence)
            else:
                pass
            print("Word level information:")
            for word_info in alternative.words:
                word = word_info.word
                start_time = word_info.start_time
                end_time = word_info.end_time
                print(
                    "\t{}s - {}s: {}".format(
                        start_time.seconds + start_time.nanos * 1e-9,
                        end_time.seconds + end_time.nanos * 1e-9,
                        word,
                    )
                )
    speech.append(z)
            
            


Processing video for speech transcription.
Alternative level information:
Transcript: 
Confidence: 0.0

Word level information:
Alternative level information:
Transcript: Until CeCe Moore founder of Entry. I'm a writer and someday when it's my turn to submit a manuscript to an agent's, I want better system entry list, simplifies how authors and agents, find each other are matching system, which is sort of like a dating website. We'll both focus and improve the odds of Landing. A book in the hands of an agent who is looking for exactly such a manuscript.
Confidence: 0.863450288772583

Word level information:
	5.1s - 5.4s: Until
	5.4s - 5.8s: CeCe
	5.8s - 6.1s: Moore
	6.6s - 7.0s: founder
	7.0s - 7.2s: of
	7.2s - 7.5s: Entry.
	8.1s - 8.2s: I'm
	8.2s - 8.3s: a
	8.3s - 8.7s: writer
	9.8s - 9.9s: and
	9.9s - 10.4s: someday
	10.4s - 10.5s: when
	10.5s - 10.7s: it's
	10.7s - 10.8s: my
	10.8s - 11.2s: turn
	11.2s - 11.3s: to
	11.3s - 11.6s: submit
	11.6s - 11.6s: a
	11.6s - 12.2s: manuscript



Processing video for speech transcription.
Alternative level information:
Transcript: Hi, I'm Gary Berman. Creator of the Cyber hero Adventures Defenders of the Digital Universe. Let me Begin by saying that I am the most reluctant person in the cyber security sector is my career has been in marketing Communications as a leading voice regarding demographic, research and Consulting until recently. I knew little about technology and even less about cybersecurity unfortunately. However, I've learned a great deal about the devastating effects that cybercrime can have on.
Confidence: 0.8134669661521912

Word level information:
	0.2s - 0.7000000000000001s: Hi,
	0.7000000000000001s - 0.9s: I'm
	0.9s - 1.2s: Gary
	1.2s - 1.8s: Berman.
	1.8s - 2.3s: Creator
	2.3s - 2.4s: of
	2.4s - 2.5s: the
	2.5s - 2.9s: Cyber
	2.9s - 3.2s: hero
	3.2s - 4.0s: Adventures
	4.1s - 4.7s: Defenders
	4.7s - 4.8s: of
	4.8s - 5.0s: the
	5.0s - 5.4s: Digital
	5.4s - 6.1s: Universe.
	6.7s - 6.8s: Let
	6.8s - 6.9s: me
	6


Processing video for speech transcription.
Alternative level information:
Transcript: More and more people are ditching processed food in animal-based products to embrace a healthy plant-based lifestyle, but having a vegan or primarily plant-based diet can be intimidating confusing, and time-consuming between all of the prep work, cook time or even knowing where to start, with all of the amazing recipes available. So what if there was an app that could connect you with fellow vegans, who love preparing healthy and delicious meals and sharing them.
Confidence: 0.901242733001709

Word level information:
	0.0s - 1.9s: More
	1.9s - 2.1s: and
	2.1s - 2.2s: more
	2.2s - 2.5s: people
	2.5s - 2.9s: are
	2.9s - 3.1s: ditching
	3.1s - 3.8s: processed
	3.8s - 4.2s: food
	4.2s - 4.5s: in
	4.5s - 5.0s: animal-based
	5.0s - 5.6s: products
	5.6s - 6.1s: to
	6.1s - 6.4s: embrace
	6.4s - 6.6s: a
	6.6s - 6.8s: healthy
	6.8s - 7.3s: plant-based
	7.3s - 8.0s: lifestyle,
	8.0s - 8.6s: but
	8.6s - 9.0s: ha


Processing video for speech transcription.
Alternative level information:
Transcript: Okay, main ft 5 in 5. Begin the Olympus and ordered a montage but I've narrowed it down.
Confidence: 0.7984685301780701

Word level information:
	2.6s - 2.9s: Okay,
	2.9s - 3.4s: main
	3.4s - 3.7s: ft
	3.7s - 3.9s: 5
	3.9s - 4.0s: in
	4.0s - 4.5s: 5.
	4.5s - 4.8s: Begin
	4.8s - 4.9s: the
	4.9s - 5.4s: Olympus
	5.4s - 5.6s: and
	5.6s - 5.9s: ordered
	5.9s - 6.0s: a
	6.0s - 6.4s: montage
	6.4s - 6.7s: but
	6.7s - 6.9s: I've
	7.0s - 7.3s: narrowed
	7.3s - 7.4s: it
	7.4s - 7.7s: down.
Alternative level information:
Transcript:  The Navy says he saved and beam levels, hepatitis of in the next five, nectar, continuing selves out by be driving.
Confidence: 0.7421154379844666

Word level information:
	9.3s - 9.4s: The
	9.4s - 9.7s: Navy
	9.7s - 10.0s: says
	10.0s - 10.0s: he
	10.0s - 10.7s: saved
	10.8s - 11.0s: and
	11.0s - 11.2s: beam
	11.2s - 11.5s: levels,
	11.5s - 12.1s: hepatitis
	12.1s - 12.3s: of
	14


Processing video for speech transcription.
Alternative level information:
Transcript: 
Confidence: 0.0

Word level information:
Alternative level information:
Transcript: This congregation bravely shared with us. What happens? The authorities, find the faithful, the police called us evil and arrested US for illegal assembly? She explains the 40 50 people, every one of them were beaten and jail and her ass
Confidence: 0.8873785138130188

Word level information:
	7.8s - 8.1s: This
	8.1s - 8.8s: congregation
	8.8s - 9.3s: bravely
	9.3s - 9.7s: shared
	9.7s - 9.9s: with
	9.9s - 10.2s: us.
	10.4s - 10.6s: What
	10.6s - 11.2s: happens?
	11.5s - 11.6s: The
	11.6s - 12.2s: authorities,
	12.2s - 12.8s: find
	12.8s - 12.9s: the
	12.9s - 13.4s: faithful,
	14.8s - 14.9s: the
	14.9s - 15.3s: police
	15.3s - 15.5s: called
	15.5s - 15.7s: us
	15.8s - 16.2s: evil
	16.2s - 16.3s: and
	16.3s - 16.7s: arrested
	16.7s - 16.8s: US
	16.8s - 17.0s: for
	17.0s - 17.5s: illegal
	17.5s - 18.1s: assembly?
	18.1


Processing video for speech transcription.
Alternative level information:
Transcript: Hi. My name is Jay and I are enrolled in the program University of Victoria Wellington. I'd like to see if you have gi Powers. Star Wars can suggest. You can't bid on to have constructed your own lightsaber. Well, this one was given to me by doorman while on holiday and I made some modifications.
Confidence: 0.764658510684967

Word level information:
	1.3s - 1.8s: Hi.
	2.0s - 2.0s: My
	2.0s - 2.2s: name
	2.2s - 2.3s: is
	2.3s - 2.7s: Jay
	3.2s - 3.4s: and
	3.4s - 3.4s: I
	4.4s - 4.5s: are
	4.5s - 4.9s: enrolled
	4.9s - 5.0s: in
	5.0s - 5.0s: the
	5.3s - 5.8s: program
	5.8s - 6.2s: University
	6.2s - 6.3s: of
	6.3s - 6.8s: Victoria
	6.8s - 7.3s: Wellington.
	9.5s - 9.6s: I'd
	9.6s - 9.8s: like
	9.8s - 9.8s: to
	9.8s - 10.0s: see
	10.0s - 10.1s: if
	10.1s - 10.2s: you
	10.2s - 10.4s: have
	10.4s - 10.7s: gi
	10.7s - 11.4s: Powers.
	12.3s - 12.6s: Star
	12.6s - 12.8s: Wars
	12.8s - 13.1s: can
	13.1s - 1


Processing video for speech transcription.
Alternative level information:
Transcript: Hi. We're the guys died 25th. I'm Anthony is the designer and the designer
Confidence: 0.8432583212852478

Word level information:
	2.3s - 2.9s: Hi.
	3.0s - 3.2s: We're
	3.2s - 3.3s: the
	3.3s - 3.6s: guys
	3.6s - 3.8s: died
	3.8s - 4.6s: 25th.
	4.7s - 4.9s: I'm
	4.9s - 5.4s: Anthony
	5.4s - 5.6s: is
	5.6s - 5.7s: the
	5.7s - 6.4s: designer
	7.0s - 7.2s: and
	7.2s - 7.3s: the
	7.3s - 8.0s: designer
Alternative level information:
Transcript:  Hey there. How you doing? My name is so much emblem, computer engineer. I'm also the software developer. We started this project in April 2013 as an experiment to see just what we were capable of doing it in the last three months. If it's really taken off, we're excited to see how far it's come. There's really so much more
Confidence: 0.7702191472053528

Word level information:
	9.8s - 9.9s: Hey
	9.9s - 10.2s: there.
	10.4s - 10.4s: How
	10.4s - 10.5s: you
	10.5s


Processing video for speech transcription.
Alternative level information:
Transcript: The way we access education is now changing. All you now need is access to a laptop and an internet connection and you can join live quality lessons taught by qualified and trainee teachers on demand all of our teachers. Go through enhanced background, checks verification of the teaching qualifications as well as enhance referencing to make sure everyone is safe online.
Confidence: 0.9128382205963135

Word level information:
	1.3s - 1.4s: The
	1.4s - 1.6s: way
	1.6s - 1.8s: we
	1.8s - 2.2s: access
	2.2s - 2.9s: education
	2.9s - 3.0s: is
	3.0s - 3.3s: now
	3.3s - 3.9s: changing.
	4.1s - 4.3s: All
	4.3s - 4.4s: you
	4.4s - 4.7s: now
	4.7s - 4.9s: need
	4.9s - 5.1s: is
	5.1s - 5.5s: access
	5.5s - 5.6s: to
	5.6s - 5.7s: a
	5.7s - 6.3s: laptop
	6.4s - 6.6s: and
	6.6s - 6.6s: an
	6.6s - 7.0s: internet
	7.0s - 7.7s: connection
	7.8s - 8.0s: and
	8.0s - 8.1s: you
	8.1s - 8.3s: can
	8.3s - 8.7s: join
	8.7s 


Processing video for speech transcription.
Alternative level information:
Transcript: 
Confidence: 0.0

Word level information:
Alternative level information:
Transcript: 
Confidence: 0.0

Word level information:


In [195]:
speech

[["Until CeCe Moore founder of Entry. I'm a writer and someday when it's my turn to submit a manuscript to an agent's, I want better system entry list, simplifies how authors and agents, find each other are matching system, which is sort of like a dating website. We'll both focus and improve the odds of Landing. A book in the hands of an agent who is looking for exactly such a manuscript.",
  ' Stop shop for all aspects of querying. So writers can focus on writing in agents, can pick up new Authors with ease, the efficient design of our website will handle submissions contain author and agent. Directories and my personal favorite part, do away with the Antiquated query letter, instead each book will be listed on a secure project page. Outlining all the important details including comps and target audience for Riders will also offer',
  " Ability report, it offers suggestions on specifically what an author can tweak to appeal to a wider array of Agents. We will also strive for the audac

In [196]:
with open("speech.txt", "w") as f:
    for item in speech:
        f.write("%s\n" % item)

